# Hyperspectral Crop Classification: XGBoost and Non-Negative Matrix Factorization

In this notebook we'll look to classify spectra of a variety of crops at various stages of development. The dataset contains spectral observations of five different crops across six stages of development taken by the Earth Observing 1 satellite from about 500 - 2300 nm at a spatial resolution of 30 meters. The crops consists of corn, cotton, rice, soybeans, and winter wheat. The six stages are (in sequential order):
- emergence / very early vegetative (`Emerge_VEarly`)
- early / mid vegetative (`Early_Mid`)
- late vegetative (`Late`)
- critical (`Critical`)
- maturing / senescence (`Mature_Senesc`)
- harvest (`Harvest`)

We'll begin by performing some exploratory analysis of our dataset, utilizing multiplicative scatter correction to look for outliers and test feature importance using MANOVA, ANOVA, and random forest. We'll then look into the use of Non-Negative Matrix Factorization (NMF) to perform dimensionality reduction. Lastly, we'll utilize XGBoost to create a machine learning classifier.

In [ ]:
import geopandas
import numpy
import pandas
import plotly
import plotly.express as px
import plotly.graph_objects as go
import re
import scipy
import sklearn

from pandas.api.types import CategoricalDtype
from tqdm.notebook import tqdm

stage_sequence = CategoricalDtype(['Emerge_VEarly', 'Early_Mid', 'Late', 'Critical', 'Mature_Senesc', 'Harvest'], ordered=True)

## Exploratory Analysis

Let's open our dataset and look at the first few entries:

In [ ]:
df = pandas.read_csv("/kaggle/input/hyperspectral-library-of-agricultural-crops-usgs/GHISACONUS_2008_001_speclib.csv")
df.head()

The spectra of each sample is contained in the columns that begin with an `X` followed by a set of digits which represent the wavelength in nanometers (nm). Let's create a variable to hold our wavelength columns and their associated values for easier reference. Let's also cast our `Stage` features to the categorical datatype we created earlier.

In [ ]:
df["Stage"] = df["Stage"].astype(stage_sequence)

wave_cols = [c for c in df.columns if re.search("X\d+", c)]
wave = numpy.array([float(x[1:]) for x in wave_cols])

Let's gather some metadata for our non-spectral columns:

In [ ]:
df.drop(columns=wave_cols).info()

`AEZ` refers to the agricultural zone which, along with the longitude and latitude, could be useful geospatial features. The `Month` attribute could be useful information as well for distinguishing between stages and crop types.

Let's go ahead and sort our data by the crop type and its stage:

In [ ]:
df = df.sort_values(by=["Crop", "Stage"], ignore_index=True)
df.drop(columns=wave_cols).head()

### Missing Data?

There is no missing data

In [ ]:
df.isna().any().any()

### Class and Stage Distribution

We have samples for five types of crop and six stages with some significant class imbalances. Corn and Soybeans are the most populous classes and cover six different stages while the remaining crops are missing some of the stages. There are significantly fewer rice samples that only cover two stages.

In [ ]:
fig = px.histogram(df, x="Crop").update_layout(yaxis_title="Count")
fig.show()

fig = px.histogram(df, x="Stage").update_layout(yaxis_title="Count")
fig.show()

fig = px.histogram(df, x="Crop", color="Stage", barmode="group").update_layout(yaxis_title="Count")
fig.for_each_trace(lambda t: t.update(name=t.name.replace('_', ' ').capitalize()))
fig.show()

In terms of AEZ there are generally only one or two crops present in our dataset, with zone 9 being the exception with 4 crop types. Corn spans 5 of 7 zones followed by cotton and soybeans at 3/7. The stages are better distributed within each of the categories.

In [ ]:
fig = px.histogram(df, x="AEZ", color="Crop").update_layout(yaxis_title="Count")
fig.for_each_trace(lambda t: t.update(name=t.name.replace('_', ' ').capitalize()))
fig.show()

fig = px.histogram(df, x="AEZ", color="Stage").update_layout(yaxis_title="Count")
fig.for_each_trace(lambda t: t.update(name=t.name.replace('_', ' ').capitalize()))
fig.show()

Crop types are fairly well spread out across the range of months, but cotton lacks samples for August while the rice samples are only found in August. The month appears to be useful in separating stages with a general progression towards later stages with time.

In [ ]:
fig = px.histogram(df, x="Month", color="Crop").update_layout(yaxis_title="Count")
fig.for_each_trace(lambda t: t.update(name=t.name.replace('_', ' ').capitalize()))
fig.show()

fig = px.histogram(df, x="Month", color="Stage").update_layout(yaxis_title="Count")
fig.for_each_trace(lambda t: t.update(name=t.name.replace('_', ' ').capitalize()))
fig.show()

Let's go ahead and create a geo-scatterplot to get an idea of where our samples were taken from:

In [ ]:
fig = px.scatter_geo(
    data_frame=df,
    lat="lat",
    lon="long",
    color="Crop",
    locationmode="USA-states",
    category_orders={"Crop": list(sorted(set(df["Crop"])))}
)
fig.update_geos(scope='usa')
fig.for_each_trace(lambda t: t.update(name=t.name.replace('_', ' ').capitalize()))
fig.update_layout(margin=dict(r=0, t=0, l=0, b=0))
fig.show()

Our samples are highly clustered, with the clusters themselves being fairly dispersed. One potential problem with using the latitude and longitude is that models could simply learn to classify based on location. Soybeans and corn are fairly well dispersed in Illinois and South Dakota while Cotton and Winter Wheat are more clustered in Texas and the same for Corn and Rice in California. In Wisconsin and Arizona we only have one crop present in our dataset.

Let's create a choropleth map of the counts in each state based on the crop and stage. To do this, we'll first need to count up the number of samples in each state, which can be done by
- Loading a geodataframe containing a polygon representation of the states
- Converting our dataframe to a geodataframe using the latitude and longitude for the point positions
- Perform a spatial join to link the two geodataframes
- Perform a pivot table operation to aggregrate the observations by class type

Let's go ahead and download a spatial file representing the US states from the US Census Bureau

In [ ]:
gdf_states = geopandas.read_file("https://www2.census.gov/geo/tiger/TIGER2021/STATE/tl_2021_us_state.zip")
#gdf_states.to_file("us_states.geojson", driver='GeoJSON')
#gdf_states = geopandas.read_file("us_states.geojson")
gdf_states.head()

Next we'll convert our crop data frame into a geodataframe in the same coordinate system as our US states shapefile and aggregate the counts across states

In [ ]:
gdf_crops = geopandas.GeoDataFrame(
    df,
    geometry=geopandas.points_from_xy(
        x=df.long,
        y=df.lat,
        crs=gdf_states.crs
    )
)

# Spatial Join
gdf_sjoin = geopandas.sjoin(gdf_states, gdf_crops)

# Pivot Table
for feature in ["Crop", "Stage"]:
    df_pivot = pandas.pivot_table(gdf_sjoin, index="GEOID", columns=[feature], aggfunc={feature: len})
    df_pivot.columns = df_pivot.columns.droplevel()
    gdf_states = gdf_states.merge(df_pivot, how='left', on='GEOID')

gdf_states.head()

Let's go ahead and visualize our crop distributions with a choropleth. Feel free to use the dropdown menu on the left to change the crop type.

In [ ]:
def get_traces_buttons(feature):
    traces = []

    categories = df[feature].unique()
    for i,c in enumerate(categories):
        trace = go.Choropleth(
            z=gdf_states[c],
            name="",
            locations=gdf_states["STUSPS"],
            locationmode="USA-states",
            colorscale="Emrld",
            colorbar_title="Count",
            zmin=0,
            zmax=(gdf_states[c].max() + 1)//1,
            visible = True if i == 0 else False)
        traces.append(trace)

    buttons = []
    for i,c in enumerate(categories):
        visible = [True if i==j else False for j,_ in enumerate(categories)]
        button = dict(
            label=c.capitalize().replace("_", " "),
            method="update",
            args=[dict(visible=visible)]
        )
        buttons.append(button)
        
    return traces, buttons

def make_choropleth(feature):
    traces, buttons = get_traces_buttons(feature)
    fig = go.Figure()
    fig.add_traces(traces)
    fig.update_layout(
        geo_scope='usa', 
        margin=dict(r=0, t=0, l=0, b=0),
        updatemenus=[dict(
            type="dropdown",
            direction="down",
            buttons=buttons,
            x=0., xanchor="left",
            y=1.1, yanchor="top")]
    )
    return fig

make_choropleth("Crop").show()
make_choropleth("Stage").show()

The crop types tend to be more geographically clustered across a small number of states while the stages tend to be spread out over a larger number of states.

### Spectra

Let's plot one of our spectra to get an idea of what it looks like

In [ ]:
px.scatter(x=wave, y=df.loc[0,wave_cols]).update_layout(xaxis_title="Wavelength (nm)", yaxis_title="Surface Reflectance (%)")

There are several gaps in the spectrum which are associated with [water vapor](https://www.e-education.psu.edu/meteo300/node/683) absorption in the atmosphere. Let's go ahead and look at how the spectra vary by their stage for each crop

In [ ]:
labels = {
    "Crop": list(sorted(df["Crop"].unique())),
    "Stage": stage_sequence.categories
}

colors = {
    "Crop": {k:px.colors.qualitative.Plotly[i] for i,k in enumerate(labels["Crop"])},
    "Stage": {k:px.colors.sample_colorscale("viridis", i/(len(labels["Stage"])+1))[0] for i,k in enumerate(labels["Stage"])}
}

def make_line_plot(feature, n:int=50):
    feature2 = [l for l in labels.keys() if l != feature][0]
    buttons = {k:[] for k in labels[feature]}
    traces = []

    for i1, c1 in enumerate(labels[feature]):
        sub = df.query(f"{feature} == '{c1}'")
        
        for i2, c2 in enumerate(sub[feature2].unique()):
            sub2 = sub.query(f"{feature2} == '{c2}'")
            if n != None:
                sub2 = sub2.sample(frac=1, random_state=11 + 17*i1).reset_index(drop=True).iloc[:n]
            
            for ir, row in sub2.iterrows():
                trace = go.Scatter(
                    name=c2,
                    legendgroup=c2,
                    x=wave,
                    y=row[wave_cols],
                    line=dict(color=colors[feature2][c2], width=0.5),
                    showlegend=True if ir == 0 else False,
                    visible=True if i1==0 else False
                )
                traces.append(trace)
                
            # Add visibility to button
            vtrue = [True for _ in range(len(sub2))]
            vfalse = [False for _ in range(len(sub2))]
            for k,v in buttons.items():
                v.extend(vtrue if k==c1 else vfalse)

    buttons = [
        dict(
            label=k.capitalize().replace("_", " "),
            method="update",
            args=[dict(visible=v)]
        )
        for k,v in buttons.items()
    ]

    fig = go.Figure()
    fig.add_traces(traces)
    fig.update_layout(
        updatemenus=[dict(
            type="dropdown",
            direction="right",
            buttons=buttons,
            x=0., xanchor="left",
            y=1, yanchor="bottom")
        ],
        yaxis_range=[0,100],
        xaxis_title="Wavelength (nm)",
        yaxis_title="Surface Reflectance (%)",
        legend_title=feature2
    )
    return fig

make_line_plot("Crop").show()
make_line_plot("Stage").show()

The different stages often have distinct patterns that could be used to separate them, such as the large amount of reflection about the 800 - 1200 nm region associated with Early mid - Critical compared to the relative absence in the earliest and latest stages.

There tends to be a lot of similarities between the different crop spectral signatures for a given stage which could hamper efforts to distinguish between them. If the stage differences aren't accounted for then one could very well train a model to classify crop types that ends up focusing on the the stage resulting in a poor performance in the real world. Luckily, the crop distribution across the different stages is fairly well balanced in this dataset.

### Outlier Detection

Let's go ahead and group our spectra by the crop type and stage and seem how similar the spectra are to one another:

In [ ]:
crops = labels["Crop"]
stages = labels["Stage"]

def get_label(crop, stage):
    return f"{crop.capitalize().replace('_', ' ')} ({stage})"

def get_traces_buttons():
    traces = []
    
    sizes = []
    categories = []
    for crop in crops:
        for stage in stages:
            # Create traces
            subset = df.query(f"Crop=='{crop}' and Stage == '{stage}'")

            if len(subset) == 0:
                continue
    
            for i, (ir, row) in enumerate(subset.iterrows()):
                trace = go.Scatter(
                    name=f"ID={ir}",
                    legendgroup=get_label(crop,stage),
                    x=wave,
                    y=row[wave_cols],
                    mode="lines",
                    marker_color=px.colors.sample_colorscale("viridis", i/len(subset))[0],
                    marker_opacity=0.5,
                    showlegend=False,
                )
                traces.append(trace)
            
            sizes.append(len(subset))
            categories.append(get_label(crop,stage))
        
    buttons = []
    for i,cat in enumerate(categories):
        vis = []
        for j,_ in enumerate(categories):
            vbool = i == j
            vtrue = [vbool for _ in range(sizes[j])]
            vis.extend(vtrue)
        buttons.append(dict(
            label=cat,
            method="update",
            args=[dict(visible=vis)]
        ))

    return traces, buttons


fig = go.Figure()

traces, buttons = get_traces_buttons()
fig.add_traces(traces)
fig.update_layout(
    yaxis_range=[0,100],
    xaxis_title="Wavelength (nm)",
    yaxis_title="Surface Reflectance (%)",
    updatemenus=[
        dict(
            type="dropdown",
            direction="down",
            buttons=buttons,
            x=0.,
            xanchor="left",
            y=1.0,
            yanchor="bottom"
        )
    ]
)

fig.for_each_trace(lambda trace: trace.update(visible=True if trace.legendgroup == get_label(crops[0], stages[0]) else False))
fig.show()

Soybean (Mature Senesc), in particular, appears to have some misclassifications with some of the spectra looking like they better aline with Soybean (Critical). There does appear to be a rather wide spread in terms of the surface reflectance while the underlying pattern looks to be about the same.

#### Multiplicative Scatter Correction (MSC)

One means of comparing spectra when there are considerable offsets in the illumance is to apply multiplicative scatter correction (MSC), which is a normalization technique that applies an offset and scale factor to align a spectrum with a reference spectrum.

Let's go ahead and try an MSC approach towards finding outliers. We'll create a reference spectra for each crop and stage group that is the median value at each wavelength and calculate the residual offset for each spectra relative to its reference spectrum.

In [ ]:
def msc_fit(data, features:list=["Crop", "Stage"]):
    """
    Performs a Multiplicative Scatter Correction (MSC) to the spectra grouped
    by the indicated columns and returns the reference spectrum along with
    the residuals.
    """
    # Compute reference spectra
    reference = df.loc[:, [*features, *wave_cols]].groupby(features).median()
    
    # Calculate residuals
    residuals = pandas.DataFrame(columns=[*features, *wave_cols])
    for index, spectrum in reference.iterrows():
        # Query database to find samples with same feature types
        subset = data.query(" and ".join(f"{c} == '{index[i]}'" for i,c in enumerate(features)))
        # Extract the spectral columns
        spectra = subset.loc[:, wave_cols]
        # Perform a linear fit
        coefs = [numpy.polyfit(spectrum, spectra.iloc[i], 1) for i in range(len(spectra))]
        # Calculate the residuals
        res = [numpy.poly1d(coefs[i])(spectrum.values) - spectra.iloc[i] for i in range(len(spectra))]
        # Turn residuals into a data frame
        res = pandas.DataFrame(res, columns=wave_cols, index=subset.index)
        # Add the feature information
        res.loc[:, features] = index
        # Update the data frame
        residuals = pandas.concat([residuals, res])
    return reference, residuals
                    
reference, residuals = msc_fit(df)

Now we can go ahead and plot the differences

In [ ]:
crops = labels["Crop"]
stages = labels["Stage"]

def get_traces_buttons():
    traces = []
    
    sizes = []
    categories = []
    for crop in crops:
        for stage in stages:
            # Create traces
            subset = residuals.query(f"Crop=='{crop}' and Stage == '{stage}'")

            if len(subset) == 0:
                continue
    
            for i, (ir, row) in enumerate(subset.iterrows()):
                trace = go.Scatter(
                    name=f"ID={ir}",
                    legendgroup=get_label(crop,stage),
                    x=wave,
                    y=row[wave_cols],
                    mode="markers",
                    marker_color=px.colors.sample_colorscale("viridis", i/len(subset))[0],
                    marker_opacity=0.5,
                    showlegend=False,
                )
                traces.append(trace)
            
            sizes.append(len(subset))
            categories.append(get_label(crop,stage))
        
    buttons = []
    for i,cat in enumerate(categories):
        vis = []
        for j,_ in enumerate(categories):
            vbool = i == j
            vtrue = [vbool for _ in range(sizes[j])]
            vis.extend(vtrue)
        buttons.append(dict(
            label=cat,
            method="update",
            args=[dict(visible=vis)]
        ))

    return traces, buttons


fig = go.Figure()

traces, buttons = get_traces_buttons()
fig.add_traces(traces)
fig.update_layout(
    xaxis_title="Wavelength (nm)",
    yaxis_title="Normalized Offset",
    updatemenus=[
        dict(
            type="dropdown",
            direction="down",
            buttons=buttons,
            x=0.,
            xanchor="left",
            y=1.0,
            yanchor="bottom"
        )
    ]
)

fig.for_each_trace(lambda trace: trace.update(visible=True if trace.legendgroup == get_label(crops[0], stages[0]) else False))
fig.show()

Quite a few of these classifications are starting to look a bit dubious with some major structural differences that likely represent a transition between stages. For example, the Soybean Late has some prominent spectral features over certain wavelength ranges, such as a smaller reflectance around 830 nm and 915 nm and some excess reflectance at 1050 nm and 1260 nm. Similarly Soybean Critical seems to have some spectral signature indicating a transition around 720 nm and 915 nm. Corn (Emerge_VEarly) has a lot of fluctuation indicating a transition from relatively high reflectance around 2100 - 2300 nm being associated with lower reflectance around 700 - 1300 nm and vice-versa. Sample 107 in particular appears to be a prominent Corn (Emerge_VEarly) outlier.

### Feature Importance

In this part we'll use a couple methods to gauge how important our features are. Additionally, it might be beneficial to normalize our data, so let's create a normalized dataset for this purpose:

In [ ]:
norm = df.loc[:,wave_cols].mean(axis=1)
df_norm = df.loc[:,wave_cols].div(norm, axis=0)
df_norm["wgt"] = norm
for c in df.columns:
    if c not in df_norm:
        df_norm[c] = df[c]

#### MANOVA

From a MANOVA test we can reject the null hypothesis that there is no difference between the average feature values across the different crops and stages across a variety of tests.

In [ ]:
from statsmodels.multivariate.manova import MANOVA

for feature in ['Crop', 'Stage']:
    manova = MANOVA.from_formula(' + '.join(c for c in [*wave_cols, "AEZ", "Month"]) + f' ~ {feature}', df)
    print(manova.mv_test())

#### Two-Way ANOVA

Since we have two classes (Crop and Stage), using a two-way ANOVA is useful for examining how important each of the features are in distinguishing between the crop and stage.

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

def get_anova(data, cols=wave_cols):
    df = {
        'Crop': pandas.DataFrame(columns=["f", "p"]),
        'Stage': pandas.DataFrame(columns=["f", "p"])
    }

    for c in cols:
        model = ols(f"{c} ~ C(Crop) + C(Stage)", data=data).fit()
        anova = sm.stats.anova_lm(model, typ=2)
        for k,v in df.items():
            v.loc[c,"f"] = anova.loc[f"C({k})"]["F"]
            v.loc[c,"p"] = anova.loc[f"C({k})"]["PR(>F)"]

    for k,v in df.items():
        v["Category"] = k
    
    return pandas.concat([*df.values()])

for norm in [False, True]:
    anova = get_anova(df_norm if norm else df, cols=[*wave_cols, "wgt"] if norm else wave_cols)
    fig = px.bar(anova, x=anova.index, y="f", color="Category", barmode="group")
    fig.update_layout(title="Normalized" if norm else "Un-Normalized")
    fig.show()

Most of the important differences seem to lie with separating the different stages rather than the different crops. Using a normalized version results in a considerable improvement in the F-score for the Stages but not much change for the Crop types. In particular, the wavelengths around 1250 nm have about a 5x increase in the F statistic using a normalized spectrum for the stage categories.

The average reflectance doesn't appear to be that important when it comes to distinguishing between the crop or stage.

#### ANOVA

Let's go ahead and perform a single ANOVA test to compare how the different features are at separating the Crop-Stage classification jointly.

In [ ]:
def get_groups(data, feature):
    categories = data[feature].unique()
    return [data.query(f"{feature} == '{c}'") for c in categories]

def get_joined(data):
    crops = get_groups(data, "Crop").copy()
    expand = []
    for c in crops:
        stages = c["Stage"].unique()
        for s in stages:
            subset = c.query(f"Stage == '{s}'")
            expand.append(subset)
    return expand

def get_anova(groups, normalized:bool=False):
    cols = [*wave_cols, "AEZ", "Month"]
    if normalized:
        cols = [*cols, "wgt"]
    anova = pandas.DataFrame(columns=["f", "p"])
    for c in cols:
        anova.loc[c] = scipy.stats.f_oneway(*[g.loc[:,c] for g in groups])
    return anova

for norm in [False, True]:
    data = df_norm if norm else df
    anova_crop = get_anova(get_groups(data, "Crop"), normalized=norm)
    anova_crop["Category"] = "Crop"
    anova_stage = get_anova(get_groups(data, "Stage"), normalized=norm)
    anova_stage["Category"] = "Stage"
    anova_both = get_anova(get_joined(data), normalized=norm)
    anova_both["Category"] = "Crop + Stage"
    anova = pandas.concat([anova_crop, anova_stage, anova_both])
    
    fig = px.bar(anova, y="f", color="Category", barmode="group", labels=dict(index="Feature", f="ANOVA F Statistic"))
    fig.update_layout(title="Normalized" if norm else "Un-Normalized")
    fig.show()

The month seems to be particularly important for separating crops and stages. It would be worth investigating whether this is truly an importance feature or if our dataset is biased in a way. Crops do have different harvest times (e.g. wheat is harvested earlier than corn), so the month would likely be an important feature to consider.

#### Random Forest

We can also use Random Forest to gauge the relative importance of each feature:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

def get_rf(y, normalized:bool=False):
    if normalized:
        X = df_norm.loc[:,[*wave_cols, "AEZ", "Month", "wgt"]]
    else:
        X = df.loc[:,[*wave_cols, "AEZ", "Month"]]

    rfc = RandomForestClassifier(n_estimators=100, random_state=9435)
    rfc.fit(X, y)
    return pandas.DataFrame(rfc.feature_importances_[None,:], columns=X.columns)

for norm in [False, True]:
    rfi_crop = get_rf(df["Crop"], normalized=norm)
    rfi_crop["Category"] = "Crop"
    rfi_stage = get_rf(df["Stage"], normalized=norm)
    rfi_stage["Category"] = "Stage"
    rfi_both = get_rf(df["Crop"].astype('str') + df["Stage"].astype('str'), normalized=norm)
    rfi_both["Category"] = "Crop + Stage"
    rfi = pandas.concat([rfi_crop, rfi_stage, rfi_both])
    
    fig = px.bar(rfi.melt(id_vars="Category"), x="variable", y="value", color="Category", barmode="group", labels=dict(variable="Feature", value="Relative Importance"))
    fig.update_layout(title="Normalized" if norm else "Un-Normalized")
    fig.show()

The results from random forest are quite different than our ANOVA tests, particularly in the importance of the month, AEZ, and average reflectance, which may be a result of ANOVA's assumptions being incorrect (e.g., constant variance across the classes) as well as the AEZ largely reducing the possible crop types to mostly one or two cases. There are some discrete clusters of importance appearing, such as around 700 nm, 915 nm, and 1200 nm indicating that some important spectral differences are likely occuring in these regions.

### Train-Test Split

Let's go ahead and create a training and testing dataset. We'll use stratified sampling, which helps preserve class balances. To ensure this applies to both the crop and stage features, we'll join the two strings together to create our class labels.

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(
    df,
    test_size = 0.33,
    random_state = 3852,
    shuffle = True,
    stratify = df["Crop"].astype(str) + df["Stage"].astype(str)
)

train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

train_norm = train.copy()
test_norm = test.copy()

for x in [train_norm, test_norm]:
    x["wgt"] = x.loc[:,wave_cols].mean(axis=1)
    x.loc[:,wave_cols] = x.loc[:,wave_cols].div(x["wgt"], axis=0)

## Dimensionality Reduction

Other than utilizing something like a convolutional neural network, inputting the full spectral feature space into a machine learning model would probably be an ill-advised decision. The large number of parameters could lead to overfitting and slow performance, while the strong correlation between neighboring wavelengths suggests we could reduce our feature space considerably. In this section we'll look at a method that is well suited for our task: non-negative matrix factorization.

### Non-Negative Matrix Factorization

One of the intriguing qualities about non-negative matrix factorization (NMF) is that it forces the decomposition to consist of a summation of non-negative terms. To illustrate the potential significance of this, consider the spectrum of a galaxy. The observed light would be a superposition of light from many different sources, such as the redder light from the old stellar population and bluer light from newly formed stars. As shown in the figure below, if one performs a PCA decomposition of the spectrum the resulting components will quickly bear little resemblence to any real spectral source and even have unphysical negative flux at certain wavelengths. NMF, however, tends to capture the underlying sources with the components corresponding to: (1) old stellar population, (2) newly-formed OB stars, (3) A-type stars, and (4-5) emission lines.

![](http://ned.ipac.caltech.edu/level5/March19/Baron/Figures/figure14.jpg)

*Decomposition of a galaxy spectra by several different methods. Figure from Vanderplas et al. (2012).*

NMF works by approximating a matrix $\bf X$ of $n$ rows (samples) and $p$ columns (features) by the product of two non-negative matrices
$$\bf
X \approx W H
$$
where $\bf W$ is of dimension $n\times c$ and $\bf H$ is $c \times p$ for some number of components $c \le \min(N,p)$

One means of choosing the number of components $c$ is to introduce a spacity score that measures the complexity of the approximation:

$$
sp(c) = \frac{n\cdot c + c\cdot p}{n \cdot p} = \frac{c}{p} + \frac{c}{n}
$$

as well as a precision score which measures the accuracy of the approximation

$$
err(c) = \frac{||{\bf X} - {\bf WH}||_{2}^{2}}{||{\bf X}||_{2}^{2}}
$$

These scores can then be averaged to create an interpretability score between 0 and 1, with smaller values indicating a more optimal balance between simplicity and accuracy:

$$
itp(c) = 0.5 \times \left[\frac{sp(c)}{\max\limits_{c\in C}sp(c)} + \frac{err(c)}{\max\limits_{c\in C}err(c)} \right]
$$

Let's go ahead and compute the precision score for several values of $c$:

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import NMF

n_components = numpy.arange(2,9,1)

def err_score(model, X, y=None):
    X_pred = model.inverse_transform(model.transform(X))
    return numpy.linalg.norm(X - X_pred)**2 / numpy.linalg.norm(X)**2
    
def compute_scores(data, n_components):
    X = data.loc[:, wave_cols]
    
    scores = []
    for n in tqdm(n_components):
        nmf = NMF(n_components=n, max_iter=1000, init='nndsvda')
        cvs = cross_val_score(nmf, X, cv=5, scoring=err_score)
        scores.append(numpy.mean(cvs))

    return scores
                      
scores = compute_scores(train_norm, n_components)

and now plot up the interpretability score for each set of components:

In [ ]:
def interp(X, n_components, error):
    sp = n_components / max(n_components)
    err = error / max(error)
    return 0.5 * (sp + err)

iscores = interp(train, n_components, scores)
fig = px.scatter(x=n_components, y=iscores)
fig.update_layout(xaxis_title="# Components", yaxis_title="CV Score (Interpretability)")
fig.show()

Looks like 5 components is our ideal number.

In [ ]:
nmf = NMF(n_components=n_components[numpy.argmin(iscores)], max_iter=1000, init='nndsvda')
nmf.fit(train_norm.loc[:,wave_cols])

Let's go ahead and plot our NMF component spectra

In [ ]:
components = nmf.components_.reshape(-1, len(wave_cols))
components = components / components.mean(axis=1, keepdims=True)

fig = go.Figure()
for i,component in enumerate(components):
    fig.add_trace(go.Scatter(x=wave, y=component, name=f"PC{i}"))
fig.update_layout(
    xaxis_title="Wavelength (nm)",
    yaxis_title="Relative Surface Reflectance",
    legend_title="Component"
)

fig.show()

Looks like there is a lot of importance features located in the 800 - 1200 nm range that our different components are trying to capture, which is where chorophyll is most active.

Let's compare our components with the average crop and stage to see if they bear any resemblance to them.

In [ ]:
def make_plot(data, feature:str):
    avg_spec = data.loc[:,[feature, *wave_cols]].groupby(feature).mean()
    
    traces = []
    # Add the averaged features
    for i in range(len(avg_spec)):
        trace = go.Scatter(x=wave, y=avg_spec.iloc[i,:], name=avg_spec.index[i].capitalize().replace("_", " "), legendgroup=feature.capitalize())
        traces.append(trace)
    
    # Add the components
    for i,component in enumerate(components):
        trace = go.Scatter(x=wave, y=component, name=f"PC{i}", visible=True if i == 0 else False, legendgroup="NMF", marker=dict(color=plotly.colors.qualitative.Plotly[len(avg_spec)]))
        traces.append(trace)

    # Buttons to make one component visible at a time
    n = len(components)
    visible = [True for _ in range(len(avg_spec))] # Keep average feature spectra visible
    visible = [[*visible, *[True if i==j else False for i in range(n)]] for j in range(n)] # Toggle components

    buttons = [
        dict(
            label=f"PC{i}",
            method="update",
            args=[dict(visible=v)]
        )
        for i,v in enumerate(visible)
    ]
    
    fig = go.Figure()
    fig.add_traces(traces)
    fig.update_layout(
        xaxis_title = "Wavelength (nm)",
        yaxis_title = "Relative Surface Reflectance",
        #legend_title = feature2,
        #yaxis_range = [0,100],
        updatemenus = [dict(
            type="buttons",
            direction="right",
            buttons=buttons,
            x=0., xanchor="left",
            y=1., yanchor="bottom"
        )]
    )

    fig.show()
    
make_plot(train_norm, "Crop")
make_plot(train_norm, "Stage")

Looks like our last component tends to capture the general shape of the averaged spectra while the remaining ones are latent features that affect the spectrum.

Let's go ahead and transform our training and testing datasets

In [ ]:
nmf_cols = [f"PC{i}" for i in range(len(nmf.components_))]
train_nmf = pandas.DataFrame(nmf.transform(train_norm.loc[:,wave_cols]), columns=nmf_cols)
test_nmf = pandas.DataFrame(nmf.transform(test_norm.loc[:,wave_cols]), columns=nmf_cols)

for c in ["AEZ", "Month", "wgt"]:
    train_nmf[c] = train_norm[c]
    test_nmf[c] = test_norm[c]

for col in ["Crop", "Stage"]:
    train_nmf[col] = train[col]
    test_nmf[col] = test[col]

and visualize the crop and stage distributions based on the NMF components

In [ ]:
labels = {
    "Crop": list(sorted(df["Crop"].unique())),
    "Stage": stage_sequence.categories
}

def make_splom(feature):
    traces = []
    cols = nmf_cols
    for fi, f in enumerate(labels[feature]):
        subset = train_nmf.query(f"{feature}=='{f}'")

        trace = go.Splom(
            name=f,
            legendgroup=f,
            dimensions=[dict(
                label=col,
                values=subset[col]
            ) for col in [*cols, feature]],
            marker=dict(color=plotly.colors.qualitative.Plotly[fi]),
            diagonal_visible=False,
            showupperhalf=False,
        )
        traces.append(trace)
        
    fig = go.Figure()
    fig.add_traces(traces)
    fig.update_layout(width=800, height=800, legend=dict(title=feature, xanchor="right", x=1))
    fig.show()

make_splom("Crop")
make_splom("Stage")

## Classifying with XGBoost

One of the most powerful machine learning methods is [XGBoost](https://arxiv.org/pdf/1603.02754.pdf), which is a tree-based learning model. While random forest focuses on creating numerous independent trees to reduce the variance and overfitting, XGBoost trains the trees sequentially such that each tree is trained to minimize the residuals from the previous tree resulting in a more powerful learner that reduces bias and underfitting. 

Tree-based learning involves the creation of a model to generate a prediction $\hat{y}_{i}$ given a set of features $\boldsymbol{x}_{i}$ associated with an observation. If there are $K$ additive functions, then the prediction is given by

$$
\hat{y}_{i} = \sum_{k=1}^{K} f_{k}(\boldsymbol{x}_i), \qquad f_k \in \mathcal{F}
$$

where $\mathcal{F} = \{f_{k}(\boldsymbol{x}) = w_{q(\boldsymbol{x})}\}(q : \mathbb{R} \rightarrow T, w \in \mathbb{R}^{T})$ is the space of regression trees, $f_{k}$ a function in the space $\mathcal{F}$, $q$ the structure of each tree, $T$ the number of leaves in each tree, and $w_i$ the weight associated with leaf $i$. The decision tree rules are specified by $q$, which are used to assign a unique leaf in each tree to the input. Finally, a weighted sum of the leaves are used to derive the final prediction. XGBoost uses additative training such that

$$
\hat{y}_{i}^{(k)} = \hat{y}_{i}^{(k-1)} + f_{k}(\boldsymbol{x}_{i})
$$

Learning the decision rules $q$ and leaf weights $w$ involves minimizing the objective loss function

$$
\mathcal{L}(y, \hat{y}) = \sum_{i=1}^{n} \ell(y_i, \hat{y}_i) + \sum_{k=1}^{K} \Omega(f_k)
$$

where the first term is the training loss (how well the model fits the data) and the second term is a regularization loss to favor simpler models over more complex ones and reduce overfitting. XGBoost uses the following regularization term

$$
\Omega(f_t) = \gamma T + \frac{1}{2}\alpha ||w||_{1} + \frac{1}{2}\lambda ||w||_{2}^2
$$

where the first part regularizes the number of of leaves $T$ and the remaining parts regularize the weights using the L1 and L2 norms respectively. Since it is not practical to search the entire space, greedy based approaches are used to construct the tree structures.

While there are a variety of parameters associated with XGBoost that need to be fine-tune for optimal results, let's keep it simple and just consider the regularization weights $\alpha$ and $\lambda$. We'll use a grid search and apply cross-validation to choose the optimal number of parameters. Let's create a function for performing this search optimization and calculating the confusion matrix:

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

def get_labels(data):
    return data.apply(lambda row: row["Crop"] + "_" + row["Stage"], axis=1)

label_encoder = LabelEncoder()
label_encoder.fit(get_labels(train_norm))

def get_input(data, wave_cols):
    X_train = data.loc[:,[*wave_cols, "AEZ", "Month", "wgt"]]
    return X_train, label_encoder.transform(get_labels(data))

def xgboost_grid_search(data, wave_cols, param_grid):
    X_train, y_train = get_input(data, wave_cols)

    search = GridSearchCV(XGBClassifier(), param_grid=param_grid, cv=3, verbose=3)
    search.fit(X_train, y_train)
    
    print(search.best_params_)
    xgb = XGBClassifier(**search.best_params_)
    xgb.fit(X_train, y_train)
    
    return xgb

def get_confusion_matrix(target, prediction):
    cmat = confusion_matrix(y_test, test_pred)

    cmat = {
        "Confusion Matrix": cmat,
        "Precision Matrix, P(Y|X)": cmat / cmat.sum(axis=0, keepdims=True), # Up+Down = P(Y|X)
        "Recall Matrix, P(X|Y)": cmat / cmat.sum(axis=1, keepdims=True) # Left+Right = P(X|Y)
    }
    
    return {k:pandas.DataFrame(v, columns=label_encoder.classes_, index=label_encoder.classes_) for k,v in cmat.items()}

### Full Spectra

For our first model let's make use of our entire spectra and later reproduce our analysis on the NMF reduced dataset.

In [ ]:
params = {
    'lambda': [0, 0.1, 1, 10],
    'alpha': [0, 0.1, 1, 10],
}
  
xgb = xgboost_grid_search(train_norm, wave_cols, params)

With our model now trained let's go ahead and test its performance

In [ ]:
X_test, y_test = get_input(test_norm, wave_cols)
test_pred = xgb.predict(X_test)

cmat = get_confusion_matrix(y_test, test_pred)

print("Accuracy: ", numpy.diag(cmat["Confusion Matrix"]).sum().sum() / cmat["Confusion Matrix"].sum().sum())

for k,v in cmat.items():
    fig = px.imshow(v)
    fig.update_layout(title=k, xaxis_title="Predicted", yaxis_title="Ground Truth", width=800, height=800)
    fig.show()

We got about 87% accuracy. Looks like the early stages of corn and soybeans are particular difficult to distinguish, which isn't that surprising. Many of the other common errors are with nearby stages of the same crop, although winter wheat and cotton seem to have a fair bit of overlap along with rice and corn.

### NMF Spectral Decomposition

Let's go ahead and redo our analysis using our NMF transformed data

In [ ]:
params = {
    'lambda': [0, 0.1, 1, 10],
    'alpha': [0, 0.1, 1, 10],
}
  
xgb_nmf = xgboost_grid_search(train_nmf, nmf_cols, params)

Looks like we are getting about 10x quicker training time, so that's a huge improvement. Let's go ahead and look at the results:

In [ ]:
X_test, y_test = get_input(test_nmf, nmf_cols)
test_pred = xgb_nmf.predict(X_test)

cmat = get_confusion_matrix(y_test, test_pred)

print("Accuracy: ", numpy.diag(cmat["Confusion Matrix"]).sum().sum() / cmat["Confusion Matrix"].sum().sum())

for k,v in cmat.items():
    fig = px.imshow(v)
    fig.update_layout(title=k, xaxis_title="Predicted", yaxis_title="Ground Truth", width=800, height=800)
    fig.show()

Our accuracy recreased by about 2% down to 85%. Many of the errors are quite similar between the two, but overall NMF appears to provide a nice approximation to our spectra.